In [ ]:
import subprocess
import networkx as nx
import matplotlib.pyplot as plt

In [31]:
def to_g6(g):
    return nx.to_graph6_bytes(g, header=False).decode("ascii").strip()

def from_g6(g6):
    return nx.from_graph6_bytes(g6.encode("ascii"))   

def geng_stream(args):
    args = ["geng", *args]
    
    proc = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    try:
        for line in proc.stdout:
            line = line.strip()
            if line:
                yield line
    finally:
        proc.stdout.close()
        proc.stderr.close()
        proc.wait()

def dedupe(graphs):
    g6s = [to_g6(g) for g in graphs]

    proc = subprocess.Popen(["shortg", "-q"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True)
    out, _ = proc.communicate("\n".join(g6s) + "\n")

    return [from_g6(ln) for ln in out.splitlines() if ln.strip()]

In [35]:
from functools import lru_cache

def _adj_bitmasks(g):
    n = g.number_of_nodes()
    if set(g.nodes()) != set(range(n)):
        G = nx.convert_node_labels_to_integers(g, ordering="sorted")
    adj = [0] * n
    for u, v in g.edges():
        adj[u] |= 1 << v
        adj[v] |= 1 << u
    return n, tuple(adj)

def is_vc(g, k):
    n, adj0 = _adj_bitmasks(g)

    @lru_cache(maxsize=None)
    def rec(adj, kk):
        if kk < 0:
            return False

        adj_list = list(adj)
        changed = True
        while changed:
            changed = False
            for u in range(n):
                du = adj_list[u].bit_count()
                if du > kk:
                    Nu = adj_list[u]
                    x = Nu
                    while x:
                        v = (x & -x).bit_length() - 1
                        adj_list[v] &= ~(1 << u)
                        x &= x - 1
                    adj_list[u] = 0
                    kk -= 1
                    if kk < 0:
                        return False
                    changed = True
                    break

        for u in range(n):
            m = adj_list[u]
            if m:
                v = (m & -m).bit_length() - 1
                break
        else:
            return True

        def take_vertex(adj_list_in, vertex):
            a = adj_list_in[:]
            Nv = a[vertex]
            x = Nv
            while x:
                w = (x & -x).bit_length() - 1
                a[w] &= ~(1 << vertex)
                x &= x - 1
            a[vertex] = 0
            return tuple(a)

        a1 = take_vertex(adj_list, u)
        if rec(a1, kk - 1):
            return True
        a2 = take_vertex(adj_list, v)
        return rec(a2, kk - 1)

    return rec(adj0, k)

In [ ]:
def is_vcf(g, k):
    if is_vc(g, k):
        return False

    for e in g.edges():
        h = g.copy()
        h.remove_edge(*e)

        if not is_vc(h, k):
            return False

    return True

def cvcf_cands(k):
    for n in range(k + 1, 2 * k + 2):
        for g6 in geng_stream([str(n), '-C', f"-D{2 * k - n + 3}"]):
            yield from_g6(g6)

def gen_cvcf(k):
    if k == 0:
        return [nx.Graph([(0, 1)])]
    return [g for g in cvcf_cands(k) if is_vcf(g, k)]

In [ ]:
cvcf = [gen_cvcf(i) for i in range(7)]

In [ ]:
from itertools import combinations_with_replacement, product, chain
from collections import Counter

def partitions(n, max_part=None, current=None):
    if max_part is None:
        max_part = n
    
    if current is None:
        current = []

    if n == 0:
        yield current.copy()
        return
    
    for i in range(min(max_part, n), 0, -1):
        current.append(i)
        yield from partitions(n - i, i, current)
        current.pop()
    
def gen_vcf(cvcf, k):
    ans = []
    
    for part in partitions(k+1):
        blocks = [list(combinations_with_replacement(cvcf[sz-1], cnt)) for sz, cnt in Counter(part).items()]
        for bs in product(*blocks):
            ans.append(nx.disjoint_union_all(chain(*bs)))
    
    return ans

vcf = [gen_vcf(cvcf, i) for i in range(7)]


In [43]:
def delete_vertex(g, v):
    h = g.copy()
    h.remove_node(v)
    return nx.convert_node_labels_to_integers(h, ordering="sorted")

def delete_edge(g, e):
    h = g.copy()
    h.remove_edge(*e)
    return nx.convert_node_labels_to_integers(h, ordering="sorted")

def contract_edge(g, e):
    h = nx.contracted_edge(g, e, self_loops=False)
    return nx.convert_node_labels_to_integers(h, ordering="sorted")

def minors(g):
    for v in g.nodes():
        yield delete_vertex(g, v)

    for e in g.edges():
        yield delete_edge(g, e)

    for e in g.edges():
        yield contract_edge(g, e)

In [44]:
def robust_extensions(forb):
    n = forb.number_of_nodes()

    for i in range(n):
        for j in range(i + 1, n):
            if not forb.has_edge(i, j):
                g = forb.copy()
                g.add_edge(i, j)
                yield nx.convert_node_labels_to_integers(g, ordering="sorted")

    for v in range(n):
        g = forb.copy()
        x = n
        g.add_node(x)
        g.add_edge(x, v)
        yield nx.convert_node_labels_to_integers(g, ordering="sorted")

    g = forb.copy()
    x, y = n, n + 1
    g.add_nodes_from([x, y])
    g.add_edge(x, y)
    yield nx.convert_node_labels_to_integers(g, ordering="sorted")

def is_avc(g, k):
    for e in g.edges():
        h = g.copy()
        h.remove_edge(*e)

        if not is_vc(h, k):
            return False

    return True

def is_avcf(g, k):
    if is_avc(g, k):
        return False

    for h in minors(g):
        if not is_avc(h, k):
            return False

    return True

def avcf_cands(vcf):
    cands = []

    for forb in vcf:
        for g in robust_extensions(forb):
            cands.append(g)

    return dedupe(cands)

def gen_avcf(vcf, k):
    cands = avcf_cands(vcf)

    return [c for c in cands if is_avcf(c, k)]


In [55]:
avcf = [gen_avcf(vcf[i], i) for i in range(7)]

In [ ]:
import os

families = {"vcf": vcf, "cvcf": cvcf, "avcf": avcf}

for name, fam in families.items():
    out_dir = os.path.join("results", name)
    os.makedirs(out_dir, exist_ok=True)

    for k, graphs in enumerate(fam):
        path = os.path.join(out_dir, f"k_{k}.g6")
        with open(path, "w", encoding="utf-8") as f:
            for g in graphs:
                if isinstance(g, str):
                    s = g.strip()
                else:
                    s = to_g6(g)
                if s:
                    f.write(s + "\n")

1478